## RECOMMENDATION SYSTEM

In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Asus\Downloads\Recommendation System\Recommendation System\anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [2]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [3]:
df.shape

(12294, 7)

In [4]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [29]:
a=df.dropna()
a

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [30]:
a.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report

In [44]:
def load_data(file_path):
    try:
        anime_df = pd.read_csv(file_path)
        print("Dataset loaded successfully!")
        print(anime_df.head())
        return anime_df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [45]:

file_path = (r"C:\Users\Asus\Downloads\Recommendation System\Recommendation System\anime.csv")
anime_df = load_data(file_path)


Dataset loaded successfully!
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  


In [57]:
def preprocess_data(df):
    print("Preprocessing data...")
    df = df.dropna(subset=['name', 'genre', 'rating'])
    df['genre'] = df['genre'].fillna('Unknown')
    df['rating'] = df['rating'].fillna(df['rating'].mean())
    print(f"Remaining rows after dropping missing data: {len(df)}")
    return df

In [58]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [59]:
anime_df = preprocess_data(anime_df)

Preprocessing data...
Remaining rows after dropping missing data: 12017


In [60]:
def combine_features(row):
    return row['genre'] + " " + str(row['rating'])

In [69]:
def extract_features(df):
    print("Extracting features...")
    df['combined_features'] = df.apply(combine_features, axis=1)
    return df

In [70]:
anime_df = extract_features(anime_df)
anime_df

Extracting features...


,anime_id,name,genre,type,episodes,rating,members,combined_features
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"Drama, Romance, School, Supernatural 9.37"
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"Action, Comedy, Historical, Parody, Samurai, S..."
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"Sci-Fi, Thriller 9.17"
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"Action, Comedy, Historical, Parody, Samurai, S..."
...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,Hentai 4.15
12290,5543,Under World,Hentai,OVA,1,4.28,183,Hentai 4.28
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,Hentai 4.88
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,Hentai 4.98


In [17]:
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(anime_df['combined_features'])
print("Feature extraction complete!")

Feature extraction complete!


In [18]:
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)
print("Cosine similarity matrix calculated!")

Cosine similarity matrix calculated!


In [73]:
def recommend_anime(anime_title, df, cosine_sim, top_n=10):
    try:
        idx = df[df['name'] == anime_title].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_indices = [i[0] for i in sim_scores[1:top_n+1]]
        return df.iloc[sim_indices][['name', 'genre', 'rating']]
    except IndexError:
        print("Anime title not found in the dataset.")
        return pd.DataFrame()
        

In [87]:
target_anime = "Naruto"
recommended_anime = recommend_anime(target_anime, anime_df, cosine_sim)
print(f"Anime similar to '{target_anime}':")
print(recommended_anime)

Anime similar to 'Naruto':
                                                   name  \
1573  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...   
486                            Boruto: Naruto the Movie   
615                                  Naruto: Shippuuden   
1103  Boruto: Naruto the Movie - Naruto ga Hokage ni...   
1343                                        Naruto x UT   
1472        Naruto: Shippuuden Movie 4 - The Lost Tower   
2458               Naruto Shippuuden: Sunny Side Battle   
2997  Naruto Soyokazeden Movie: Naruto to Mashin to ...   
7867                                    Iron Virgin Jun   
1930                                  Dragon Ball Super   

                                                  genre  rating  
1573  Action, Comedy, Martial Arts, Shounen, Super P...    7.50  
486   Action, Comedy, Martial Arts, Shounen, Super P...    8.03  
615   Action, Comedy, Martial Arts, Shounen, Super P...    7.94  
1103  Action, Comedy, Martial Arts, Shounen, Super P...    

In [90]:
def create_labels(df, rating_threshold=9.2):
    df['relevant'] = df['rating'].apply(lambda x: 1 if x >= rating_threshold else 0)
    return df

In [91]:
anime_df = create_labels(anime_df)
print(anime_df[['name', 'rating', 'relevant']].head())

                               name  rating  relevant
0                    Kimi no Na wa.    9.37         1
1  Fullmetal Alchemist: Brotherhood    9.26         1
2                          Gintama°    9.25         1
3                       Steins;Gate    9.17         0
4                     Gintama&#039;    9.16         0


In [23]:
train_df, test_df = train_test_split(anime_df, test_size=0.2, random_state=42)

In [24]:
def get_recommendation_labels(test_df, train_df, cosine_sim, threshold=0.5):
    y_true = []
    y_pred = []

In [25]:
for _, row in test_df.iterrows():
        anime_title = row['name']
        true_label = row['relevant']

In [26]:
def get_recommendation_labels(test_df, train_df, cosine_sim, threshold=0.5):
    y_true = []  
    y_pred = [] 

    for _, row in test_df.iterrows():
        anime_title = row['name']
        true_label = row['relevant']  

        try:
            if anime_title in train_df['name'].values:
                idx = train_df[train_df['name'] == anime_title].index[0]
                recommended = [
                    1 if sim_score[1] >= threshold else 0 for sim_score in sim_scores[1:11]
                ]
                y_true.append(true_label)
                y_pred.append(1 if sum(recommended) > 0 else 0)  
            else:
                y_true.append(true_label)
                y_pred.append(0)

        except Exception as e:
            print(f"Error processing {anime_title}: {e}")
            y_true.append(true_label)
            y_pred.append(0)  

    return y_true, y_pred

In [27]:
from sklearn.metrics import classification_report

y_true, y_pred = get_recommendation_labels(test_df, train_df, cosine_sim)

print("Classification Report:")
print(classification_report(y_true, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.67      1.00      0.80      1609
           1       0.00      0.00      0.00       795

    accuracy                           0.67      2404
   macro avg       0.33      0.50      0.40      2404
weighted avg       0.45      0.67      0.54      2404



C:\Users\Asus\Downloads\Telegram Desktop\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Asus\Downloads\Telegram Desktop\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Asus\Downloads\Telegram Desktop\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

#Item-Based Collaborative Filtering:
Recommends items based on the similarity between users.
Users who have similar preferences will like similar items.

#Item-Based Collaborative Filtering:
Recommends items based on the similarity between items.
Items that are similar (frequently rated/purchased together) will appeal to users.

#Collaborative Filtering is a popular recommendation technique that predicts user preferences based on historical interactions between users and items. It relies on the idea that users who have agreed in the past will agree again in the future.

Input: A user-item interaction matrix, where rows represent users and columns represent items (e.g., movies, products, songs). Entries in the matrix represent:

Explicit feedback: Ratings (e.g., 1-5 stars).
Implicit feedback: Interactions like purchases, clicks, or views.